In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.env.dunderbot_env import DunderBotEnv

loading config from config/config.yml


# TODO:


- load interesting data, specified in config
- own observations (look at https://github.com/notadamking/RLTrader for (kinda) readable examples)
    - remove the bias when getting the observations (suggestion in the repo issues)
- understand the rendering, especially during learning
- understand the time steps in learn and predict
- wrap running code in function in main.py (to enable more dynamic data loading and train/test)

- add more detailed actions for buy and sell amounts (still Discrete, I believe)
- setup train/test functionality (separate reset functions?)

# DONE: 
- add config
- modularize rewards
- own actions

# Run main

In [3]:
import gym
import json
import datetime as dt

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
from src.env.dunderbot_env import DunderBotEnv

import pandas as pd

df = pd.read_csv('./data/input/AAPL.csv')
df = df.sort_values('Date')
df = df.drop(columns=['Unnamed: 0'])

# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: DunderBotEnv(df)])

In [5]:
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=20000, log_interval=10)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



---------------------------------------
| approxkl           | 1.6484948e-10  |
| clipfrac           | 0.0            |
| explained_variance | -7.15e-07      |
| fps                | 184            |
| n_updates          | 1              |
| policy_entropy     | 1.0986121      |
| policy_loss        | -1.0638032e-06 |
| serial_timesteps   | 128            |
| time_elapsed       | 8.51e-05       |
| total_timesteps    | 128            |
| value_loss         | 97045544.0     |
---------------------------------------
--------------------------------------
| approxkl           | 0.00039071415 |
| clipfrac           | 0.0           |
| explained_variance | -4.02e-05     |
| fps                | 348           |
| n_updates          | 10            |
| policy

--------------------------------------
| approxkl           | 0.000269819   |
| clipfrac           | 0.0           |
| explained_variance | 0             |
| fps                | 352           |
| n_updates          | 90            |
| policy_entropy     | 0.8059322     |
| policy_loss        | 0.00089131424 |
| serial_timesteps   | 11520         |
| time_elapsed       | 31.9          |
| total_timesteps    | 11520         |
| value_loss         | 2965985500.0  |
--------------------------------------
--------------------------------------
| approxkl           | 2.647382e-05  |
| clipfrac           | 0.0           |
| explained_variance | 2.38e-07      |
| fps                | 367           |
| n_updates          | 100           |
| policy_entropy     | 0.99140143    |
| policy_loss        | 0.00055694976 |
| serial_timesteps   | 12800         |
| time_elapsed       | 35.4          |
| total_timesteps    | 12800         |
| value_loss         | 5112683000.0  |
-------------------------

In [6]:
obs = env.reset()
for i in range(2000):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if i % 100 == 0:
        env.render()

Step: 3071
Balance: 10000.0
Shares held: 0 (Total sold: 0)
Avg cost for held shares: 0 (Total sales value: 0.0)
Net worth: 10000.0 (Max net worth: 10000)
Profit: 0.0
Step: 3171
Balance: 10044.468480323076
Shares held: 0 (Total sold: 70)
Avg cost for held shares: 0 (Total sales value: 17332.56375506134)
Net worth: 10044.468480323076 (Max net worth: 10152.355958769105)
Profit: 44.46848032307571
Step: 3271
Balance: 10364.008734699888
Shares held: 0 (Total sold: 120)
Avg cost for held shares: 0 (Total sales value: 32539.139377646377)
Net worth: 10364.008734699888 (Max net worth: 10369.538262211645)
Profit: 364.008734699888
Step: 3371
Balance: 10204.367444071575
Shares held: 0 (Total sold: 210)
Avg cost for held shares: 0 (Total sales value: 63426.8299518705)
Net worth: 10204.367444071575 (Max net worth: 10474.252789244725)
Profit: 204.36744407157494
Step: 3471
Balance: 9860.326521278334
Shares held: 0 (Total sold: 250)
Avg cost for held shares: 0 (Total sales value: 78241.8005506619)
Net w